<img src="./pictures/logo_sizinglab.png" style="float:right; max-width: 60px; display: inline" alt="SizingLab" /></a>


## Dynamic models for propellers
*Written by Marc Budinger, Aitor Ochotorena (INSA Toulouse) and Scott Delbecq (ISAE-SUPAERO), Toulouse, France.*


The complexity of the propellers characteristics increases as they work in dynamic scenarios. So far, we have seen how $C_T$ and $C_P$ coefficients can be well expressed in terms of geometric ratio (pitch/diameter). In these dynamic scenarios, these coefficients depend also on the speed.  Based on dimensional analysis and the polynomial regression of suppliers data, adequate prediction models for $C_T$ and $C_P$ are generated. 


*Flow distribution in multirotor vehicle for climbing*
![Climbing](./pictures/drone_axial_vertical_flight.png)


### Data analysis

Based on the datasheets of different models of *APC propellers MR*, estimation models are developed to characterize the thrust and power coefficients of the propellers for dynamic scenarios. In the next DataFrame, an extra column is added to calculate the value of the dimensionless air compressibility estimation or B coefficient.

In [ ]:
import pandas as pd
import math

# Read the .csv file with bearing data
path='./docs/'
df = pd.read_csv(path+'Propeller_Data.csv', sep=';')
K=101000 #[Pa] bulk modulus air
df['B coef']=1.18*(df['RPM']/60*2*3.14)**(2)*(df['DIAMETER_in']*0.0254)**(2)/K

# Print the head (first lines of the file)
df.head()

### Heatmap

The use of Heatmaps enables to visualize numerically the dependence of the parameters on each others to describe their behaviour. A value close to '1' will mean a very strong dependence, while a value close to '0' means a negligible effect. We see how $Cp$ and $Ct$ do not longer depend only on beta to describe their behaviour. A tutorial of seaborn heatmap can be found [here](https://likegeeks.com/seaborn-heatmap-tutorial/).

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

f, axes = plt.subplots(1,2,figsize=(12,4))

df_Cp=df[['Cp','BETA','J', 'B coef']]
df_Ct=df[['Ct','BETA','J', 'B coef']]

# heatmap
sns.heatmap(df_Cp.corr(method='pearson')**2,annot=True,cmap='coolwarm',square=True, ax=axes[0])
sns.heatmap(df_Ct.corr(method='pearson')**2,annot=True,cmap='coolwarm',square=True, ax=axes[1])



### Scatter matrix

Another technique very similar to the heatmap to observe dependencies is the scatter matrix, in which we can represent in a matrix small subplots. Here we are interested in studying the trend of the parameter rather than knowing the exact value of the magnitude.

In [ ]:
pd.plotting.scatter_matrix(df_Cp)
pd.plotting.scatter_matrix(df_Ct)
plt.tick_params(axis = 'both', labelsize = 14)
plt.rcParams.update({'font.size': 14})


### Surrogate modeling techniques

In this part, data are fitted using a non-linear polynomial model regression. For this purpose, a polynomial $(beta+ J)^n$ is developed using the package `PolynomialFeatures` of [scikit learn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) to a degree n, whose value of determination coefficient ($R^2$) is higher than required.
As a condition, we set that the value of R squared required must be $R_{est}=0.98$
At the end, data and the surrogate model are plotted in a 3D graph.

- $C_p=f(\beta,J)$

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np

R_est=0.98 # minimum R-squared value to achieve

%matplotlib notebook 
# read dataframe values
X = df[['J', 'BETA']].values
y1 = df['Cp'].values

# extracting columns from X
mu_col=X[:,0] 
beta_col=X[:,1]

#a matrix of mu and beta is made
XX = np.concatenate((mu_col.reshape(np.size(y1),1),beta_col.reshape(np.size(y1),1)),axis=1)

# in this while we increase the polynomial binomium until R2 reaches the limit
i=1 # start by 1.Grade polynomium 
while True:
    poly = PolynomialFeatures(i) # development of i-grade polynomium
    x_poly = poly.fit_transform(XX)

    model = sm.OLS(y1, x_poly) #fit regression model to the data
    result= model.fit() # save value of coeficients
    y_poly_pred = result.predict(x_poly) # according to the regression model found, a value of y is generated

    rmse = np.sqrt(mean_squared_error(y1,y_poly_pred)) #Root mean square error
    r2 = r2_score(y1,y_poly_pred) #R2 correlation value
    i +=1 # we increase the polynomium degre
    if r2>R_est: #if R2 calculated is greater than 
        break
print("RMSE=",rmse)
print("R2=",r2)

#here we expand the formula of the i-degree polynomium
data = pd.DataFrame.from_dict({
    'J': np.random.randint(low=1, high=10, size=5),
    'BETA': np.random.randint(low=-1, high=1, size=5),
})

p = PolynomialFeatures(degree=i-1).fit(data)

#concatenate the OLS regression parameters with variables 
string1=['+({:.3f})*'.format(x) for x in result.params]
string2=p.get_feature_names(data.columns)

Result=""
for i in range(len(string1)):
        Result += string1[i]
        Result += string2[i]
print('Cp=',Result)


#meshgrid 3D for plot
xx1, xx2 = np.meshgrid(np.linspace(mu_col.min(), mu_col.max(), len(y1)), 
                       np.linspace(beta_col.min(), beta_col.max(), len(y1)))


#create the 3D axis 
fig = plt.figure(figsize=(10, 8))
ax = Axes3D(fig, azim=-210, elev=15)


ax.scatter(mu_col, beta_col, y1, color='black', alpha=1.0, facecolor='white')

# reshape data to suit in the grid. Here a x b = len(y1)
xp = np.reshape(mu_col, (94, 30))
yp = np.reshape(beta_col, (94, 30))
zp = np.reshape(y_poly_pred, (94, 30))

# plotting the predicted surface
ax.plot_surface(xp, yp, zp, cmap=plt.cm.RdBu_r, alpha=0.3, linewidth=0)

#axis labels
ax.set_xlabel('J (Advance ratio) [-]',fontsize=14)
ax.set_ylabel('angle (pitch/diameter) [-]',fontsize=14)
ax.set_zlabel(r'$C_P [-]$',fontsize=14)

plt.grid()

plt.tick_params(axis = 'both', labelsize = 14)

- $C_t=f(\beta,J)$

In [ ]:
R_est=0.98 # minimum R-squared value to achieve

# read dataframe values
X = df[['J', 'BETA']].values
y1 = df['Ct'].values

# extracting columns from X
mu_col=X[:,0] 
beta_col=X[:,1]

#a matrix of mu and beta is made
XX = np.concatenate((mu_col.reshape(np.size(y1),1),beta_col.reshape(np.size(y1),1)),axis=1)

# in this while we increase the polynomial binomium until R2 reaches the limit
i=1 # start by 1.Grade polynomium 
while True:
    poly = PolynomialFeatures(i) # development of i-grade polynomium
    x_poly = poly.fit_transform(XX)

    model = sm.OLS(y1, x_poly) #fit regression model to the data
    result= model.fit() # save value of coeficients
    y_poly_pred = result.predict(x_poly) # according to the regression model found, a value of y is generated

    rmse = np.sqrt(mean_squared_error(y1,y_poly_pred)) #Root mean square error
    r2 = r2_score(y1,y_poly_pred) #R2 correlation value
    i +=1 # we increase the polynomium degre
    if r2>R_est: #if R2 calculated is greater than 
        break
print("RMSE=",rmse)
print("R2=",r2)

#here we expand the formula of the i-degree polynomium
data = pd.DataFrame.from_dict({
    'mu': np.random.randint(low=1, high=10, size=5),
    'beta': np.random.randint(low=-1, high=1, size=5),
})

p = PolynomialFeatures(degree=i-1).fit(data)

#concatenate the OLS regression parameters with variables 
string1=['+({:.3f})*'.format(x) for x in result.params]
string2=p.get_feature_names(data.columns)

Result=""
for i in range(len(string1)):
        Result += string1[i]
        Result += string2[i]
print('Ct=',Result)


#meshgrid 3D for plot
xx1, xx2 = np.meshgrid(np.linspace(mu_col.min(), mu_col.max(), len(y1)), 
                       np.linspace(beta_col.min(), beta_col.max(), len(y1)))


#create the 3D axis 
fig = plt.figure(figsize=(10, 8))
ax = Axes3D(fig, azim=-210, elev=15)


ax.scatter(mu_col, beta_col, y1, color='black', alpha=1.0, facecolor='white')

# reshape data to suit in the grid. Here a x b = len(y1)
xp = np.reshape(mu_col, (94, 30))
yp = np.reshape(beta_col, (94, 30))
zp = np.reshape(y_poly_pred, (94, 30))

# plotting the predicted surface
ax.plot_surface(xp, yp, zp, cmap=plt.cm.RdBu_r, alpha=0.3, linewidth=0)

#axis labels
ax.set_xlabel('J (Advance ratio) [-]',fontsize=14)
ax.set_ylabel('angle (pitch/diameter) [-]',fontsize=14)
ax.set_zlabel(r'$C_T [-]$',fontsize=14)

plt.grid()

plt.tick_params(axis = 'both', labelsize = 14)


### Extra: values of $C_T$ and $C_P$ for other propeller models

Curves of thrust and power coefficient are represented for the dynamic behaviour for other manufacturer's models. Required aerodynamic thrust and power fall as the propeller gains speed. Performance efficiency has a parabolic form.

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

path='./docs/'
df = pd.read_csv(path+'9x45-DynamicProps.csv', sep=';')

f, ((ax1, ax2,ax3)) = plt.subplots(1, 3,figsize=(10, 5))
f.tight_layout()

groups = df.groupby('TYPE')

for name, group in groups:
    ax1.plot(group.J, group.CT, marker='o', linestyle='', ms=5, 
label=name)
    
#ax1.set_title('CT vs J')
ax1.set_xlabel('J[-]',fontsize=14)
ax1.set_ylabel(r'$C_T[-]$',fontsize=14)

for name, group in groups:
    ax2.plot(group.J, group.CP, marker='o', linestyle='', ms=5, 
label='_nolegend_')
    
#ax2.set_title('CP vs J')
ax2.set_xlabel('J[-]',fontsize=14)
ax2.set_ylabel(r'$C_P[-]$',fontsize=14)

for name, group in groups:
    ax3.plot(group.J, group.Eta, marker='o', linestyle='', ms=5, 
label='_nolegend_')
    
#ax3.set_title(r'Efficiency $\eta$ vs Advance Ratio J')
ax3.set_xlabel('J[-]',fontsize=14)
ax3.set_ylabel(r'$\eta[-]$',fontsize=14)

f.legend(loc='lower center', bbox_to_anchor=(0.8, 0.05),prop={'size': 14},ncol=4)
#f.legend(loc=(axbox.x0,0),prop={'size': 8})
ax1.grid()    

ax2.grid() 


ax3.grid()    

ax1.tick_params(axis = 'both', labelsize = 14)
ax2.tick_params(axis = 'both', labelsize = 14)
ax3.tick_params(axis = 'both', labelsize = 14)

f.tight_layout()
#plt.subplots_adjust(left=None, bottom=None, right=None, top=None,
#                wspace=None, hspace=None)


### References

- Principles of Helicopter Aerodynamics. Leishman
- [MR Propellers](https://www.masterairscrew.com/pages/mr-drone-propellers)
- [Drones](https://www.droneomega.com/quadcopter-propeller/)
